# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **rostock**, Germany.

Since there are lots of restaurants in rostock we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of rostock center will be obtained using **Google Maps API geocoding** of well known rostock location (Alexanderplatz)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around rostock city center.

Let's first find the latitude & longitude of rostock city center, using specific, well known address and Google Maps geocoding API.

In [1]:
import folium
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import geocoder
import pgeocode

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# read csv file
df = pd.read_csv('data/DE.txt', delimiter="\t", names=['country-code', 'postal-code', 'city',
                 'state', "state-abbrev", "region", "r-code", "province", "p-code", "latitude", "longitude", "accuracy"], encoding='utf-8')

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values) == True]

# drop rows where province does not include Rostock
df = filter_rows_by_values(df, "province", ["Kreisfreie Stadt Rostock", "Landkreis Rostock"])

df.head()

# df.shape


,country-code,postal-code,city,state,state-abbrev,region,r-code,province,p-code,latitude,longitude,accuracy
6251,DE,17166,Groß Wokern,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7513,12.4925,4.0
6252,DE,17166,Bristow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.7123,12.6257,4.0
6253,DE,17166,Dalkendorf,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8148,12.5021,4.0
6254,DE,17166,Bülow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.6899,12.6007,4.0
6255,DE,17166,Alt Sührkow,Mecklenburg-Vorpommern,MV,NaN,0.0,Landkreis Rostock,13072.0,53.8024,12.6512,4.0


In [3]:
# get Rostock geo-location
address = 'Rostock, Germany'
geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
rostock_center = [latitude, longitude]
print(
    f"The geograpical coordinates of Rostock Germnay are {latitude}, {longitude}.")


The geograpical coordinates of Rostock Germnay are 54.0924445, 12.1286127.


In [4]:
# create map
map_rostock = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, city in zip(df['latitude'], df['longitude'], df['city']):
    label = f"{city}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_rostock)
map_rostock


In [5]:
import pyproj

import math


def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong', datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]


def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong', datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]


def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)


In [8]:
from constants import GOOGLE_API_KEY

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?key={api_key}&latlng={latitude},{longitude}"
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None


addr = get_address(GOOGLE_API_KEY, rostock_center[0], rostock_center[1])
print('Reverse geocoding check')
print('-----------------------')
print(f"Address of [{rostock_center[0]}, {rostock_center[1]}] is: {addr}")


Reverse geocoding check
-----------------------
Address of [54.0924445, 12.1286127] is: Stadthafen 71, 18057 Rostock, Germany


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(df['latitude'], df['longitude']):
    address = get_address(GOOGLE_API_KEY, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    # We don't need country part of address
    address = address.replace(', Germany', '')
    addresses.append(address)
    print(' .', end='')
print(' done.')


Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [13]:
(addresses)

['Dorfstraße 28, 17166 Groß-Wokern',
 'Gutshof 10, 17166 Schorssow',
 'Hauptstraße 3, 17166 Dalkendorf',
 'Seestraße 4, 17166 Schorssow',
 'Dorfstraße 16, 17166 Alt Sührkow',
 'Dorfstraße 22, 17166 Hohen-Demzin',
 'Nördliche Ringstraße 41, 17166 Teterow',
 'Pösel 17, 17166 Groß Roge',
 'Philosophenweg 2, 17166 Dahmen',
 'Alte Dorfstraße 2, 17168 Jördenstorf',
 'Dorfstraße 64, 17168 Groß Wüstenfelde',
 'Poggelow 16, 17168 Schwasdorf',
 'Unnamed Road, 17168 Sukow-Levitzow',
 'Schloßweg 8, 17168 Lelkendorf',
 'Dorfstraße 8A, 17168 Thürkow',
 'Schlossallee 1, 17168 Groß Wüstenfelde',
 'Parkweg 13, 17168 Thürkow',
 'Hauptstraße 12, 17168 Prebberede',
 'Dörpstrat 3, 17168 Warnkenhagen',
 'Remlin 28, 17168 Schwasdorf',
 'Kastanienallee 17, 17168 Prebberede',
 'Dorfstraße 11A, 17179 Lühburg',
 'Markt 11, 17179 Gnoien',
 'Darguner Str. 3, 17179 Altkalen',
 'Wasdow, 17179 Wasdow',
 'Dorfstraße 33A, 17179 Walkendorf',
 'An der Landstraße 40, 17179 Behren-Lübchin',
 'Finkenthal, 17179 Finkenthal',

In [14]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': df["latitude"],
                             'Longitude': df["longitude"]
                             })

df_locations.head(10)


,Address,Latitude,Longitude
6251,"Dorfstraße 28, 17166 Groß-Wokern",53.7513,12.4925
6252,"Gutshof 10, 17166 Schorssow",53.7123,12.6257
6253,"Hauptstraße 3, 17166 Dalkendorf",53.8148,12.5021
6254,"Seestraße 4, 17166 Schorssow",53.6899,12.6007
6255,"Dorfstraße 16, 17166 Alt Sührkow",53.8024,12.6512
6256,"Dorfstraße 22, 17166 Hohen-Demzin",53.7214,12.5438
6257,"Nördliche Ringstraße 41, 17166 Teterow",53.7754,12.5749
6258,"Pösel 17, 17166 Groß Roge",53.7785,12.5047
6259,"Philosophenweg 2, 17166 Dahmen",53.6609,12.5814
6260,"Alte Dorfstraße 2, 17168 Jördenstorf",53.8782,12.6162


In [15]:
df_locations.to_pickle('data/locations.pkl')


In [18]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [27]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

from constants import FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET

def get_info(lats, lons):

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(
            lat, lon, food_category, FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET, radius=40000, limit=100)
        print(' .', end='')
    print(' done.')
    return venues


# Try to load from local file system in case we did this before
venues = {}
loaded = False
try:
    with open('venues.pkl', 'rb') as f:
        venues = pickle.load(f)
    print('Venues data loaded.')
    if len(venues > 0):
        loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    venues = get_info(df["latitude"], df["longitude"])
    
    # Let's persists this in local file system
    with open('venues.pkl', 'wb') as f:
        pickle.dump(venues, f)
        


Venues data loaded.
Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [31]:
import numpy as np

print('Total number of venues:', len(venues))
venues

Total number of venues: 80


[('4bee9e0c3686c9b645b0246e',
  'Borwin Hafenrestaurant',
  [('Seafood Restaurant', '4bf58dd8d48988d1ce941735')],
  (54.09253072737255, 12.137637341081113),
  'Am Strande 2, 18055 Rostock',
  18875),
 ('4bf975e3508c0f47924d3f31',
  'Likörfabrik',
  [('Restaurant', '4bf58dd8d48988d1c4941735')],
  (54.08736433910524, 12.1439671971938),
  'Grubenstr. 1, 18055 Rostock',
  18168),
 ('592ab73d0802d4253224553d',
  'Jan & Mitch Café Bar',
  [('Café', '4bf58dd8d48988d16d941735')],
  (54.089382, 12.134797),
  'Breite Str. 12-15, 18055 Rostock',
  18721),
 ('4d9728eea2c654818e12d153',
  'Café Heumond',
  [('Café', '4bf58dd8d48988d16d941735')],
  (54.08591386877662, 12.135787203202261),
  'Hermannstraße 36, 18055 Rostock',
  18384),
 ('4c0147a9783720a18265c62e',
  'Zum Alten Fritz',
  [('German Restaurant', '4bf58dd8d48988d10d941735')],
  (54.0921770557118, 12.127036196812176),
  'Warnowufer 65, 18057 Rostock',
  19285),
 ('4db1a839cda14b702a4a19d3',
  'Rosmarino',
  [('Mediterranean Restaurant', 

In [33]:
map_rostock = folium.Map(location=rostock_center, zoom_start=13)
folium.Marker(rostock_center, popup='Alexanderplatz').add_to(map_rostock)
for v in venues:
    color = 'blue'
    folium.CircleMarker(v[3], radius=3, color=color, fill=True,
                        fill_color=color, fill_opacity=1).add_to(map_rostock)
map_rostock


In [6]:
rostock_center_x, rostock_center_y = lonlat_to_xy(
    rostock_center[1], rostock_center[0])  # City center in Cartesian coordinates

k = math.sqrt(3) / 2  # Vertical offset for hexagonal grid cells
x_min = rostock_center_x - 6000
x_step = 600
y_min = rostock_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i % 2 == 0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(
            rostock_center_x, rostock_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packag

364 candidate neighborhood centers generated.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  app.launch_new_instance()


In [7]:
map_rostock = folium.Map(location=rostock_center, zoom_start=13)
folium.Marker(rostock_center, popup='Rostock').add_to(map_rostock)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rostock)
    folium.Circle([lat, lon], radius=300, color='blue',
                  fill=False).add_to(map_rostock)
    #folium.Marker([lat, lon]).add_to(map_rostock)
map_rostock


In [ ]:
# get the geo-location of each post-code
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis(
            f"{postal_code}, Rostock, Germnay")
        lati_long_coords = g.latlng
    return lati_long_coords


# test
get_latilong('18069')


In [ ]:
from constants import FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET
# Foursquare API config
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print(f"CLIENT_ID: {FOURSQUARE_CLIENT_ID}")
print(f"CLIENT_SECRET: {FOURSQUARE_CLIENT_SECRET}")


Your credentails:
CLIENT_ID: MZUGNTHUN4CZJ1BURHWGXXXO0XYGMRNDRWB2KADTMQYBKYHZ
CLIENT_SECRET: MTQVYA102ORA4YZVXD5J1Z200IZMAFMRVKRP4AESXUO0G0K5
